# **Churn Prediction**

In [1]:
import pyodbc as odbc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import joblib

## **Connecting to SQL Server**

In [2]:
# Define your connection
conn = odbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=MRSTOCK\HUMBER_DB;'
    r'DATABASE=db_Churn;'
    r'Trusted_Connection=yes;'
)

# Write your SQL query (view name)
query = "SELECT * FROM [db_Churn].[dbo].[vw_ChurnData]"

# Load the view into a DataFrame
df = pd.read_sql(query, conn)

# Preview
print(df.head())

# Always good practice to close connection
conn.close()

C:\Users\shyam\AppData\Local\Temp\ipykernel_26476\3184914769.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


  Customer_ID  Gender  Age Married           State  Number_of_Referrals  \
0   11098-MAD  Female   30     Yes  Madhya Pradesh                    0   
1   11114-PUN    Male   51      No          Punjab                    5   
2   11167-WES  Female   43     Yes     West Bengal                    3   
3   11179-MAH    Male   35      No     Maharashtra                   10   
4   11180-TAM    Male   75     Yes      Tamil Nadu                   12   

   Tenure_in_Months Value_Deal Phone_Service Multiple_Lines  ...  \
0                31     Deal 1           Yes             No  ...   
1                 9     Deal 5           Yes             No  ...   
2                28     Deal 1           Yes            Yes  ...   
3                12       None           Yes             No  ...   
4                27     Deal 2           Yes             No  ...   

    Payment_Method Monthly_Charge Total_Charges Total_Refunds  \
0  Bank Withdrawal      95.099998   6683.399902          0.00   
1  Bank Wi

## **Data Preprocessing**

In [6]:
df['Customer_Status'].value_counts()

Customer_Status
Stayed     4275
Churned    1732
Name: count, dtype: int64

In [7]:
# Drop columns that won't be used for prediction
data = df.drop(['Customer_ID', 'Churn_Category', 'Churn_Reason'], axis=1)


In [8]:
# List of columns to be label encoded
columns_to_encode = [
    'Gender', 'Married', 'State', 'Value_Deal', 'Phone_Service', 'Multiple_Lines',
    'Internet_Service', 'Internet_Type', 'Online_Security', 'Online_Backup',
    'Device_Protection_Plan', 'Premium_Support', 'Streaming_TV', 'Streaming_Movies',
    'Streaming_Music', 'Unlimited_Data', 'Contract', 'Paperless_Billing',
    'Payment_Method'
]

# Encode categorical variables except the target variable
label_encoders = {}
for column in columns_to_encode:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

In [9]:
# Manually encode the target variable 'Customer_Status'
data['Customer_Status'] = data['Customer_Status'].map({'Stayed': 0, 'Churned': 1})

In [10]:
# Split data into features and target
X = data.drop('Customer_Status', axis=1)
y = data['Customer_Status']

In [11]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
data

,Gender,Age,Married,State,Number_of_Referrals,Tenure_in_Months,Value_Deal,Phone_Service,Multiple_Lines,Internet_Service,Internet_Type,Online_Security,Online_Backup,Device_Protection_Plan,Premium_Support,Streaming_TV,Streaming_Movies,Streaming_Music,Unlimited_Data,Contract,Paperless_Billing,Payment_Method,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status
0,0,30,1,11,0,31,0,1,0,1,2,1,1,0,1,0,1,1,1,2,0,0,95.099998,6683.399902,0.00,0,631.719971,7315.120117,0
1,1,51,0,15,5,9,4,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,49.150002,169.050003,0.00,10,122.370003,301.420013,1
2,0,43,1,21,3,28,0,1,1,1,2,1,1,1,1,1,1,1,0,2,1,0,116.050003,8297.500000,42.57,110,1872.979980,10237.910156,0
3,1,35,0,12,10,12,5,1,0,1,1,1,1,1,1,1,1,1,1,2,1,1,84.400002,5969.299805,0.00,0,219.389999,6188.689941,0
4,1,75,1,17,12,27,1,1,0,1,1,1,0,0,1,1,1,0,0,2,1,1,72.599998,4084.350098,0.00,140,332.079987,4556.430176,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002,0,43,0,11,3,19,4,1,0,1,1,0,0,0,0,0,0,0,1,1,1,1,43.450001,345.500000,0.00,0,54.240002,399.739990,0
6003,0,53,1,10,7,10,4,1,0,1,1,0,0,0,0,1,0,0,1,0,0,1,55.500000,227.350006,0.00,0,155.839996,383.190002,0
6004,0,46,0,12,7,10,5,1,0,1,2,0,0,0,0,0,1,1,1,0,1,0,78.099998,1122.400024,0.00,0,62.299999,1184.699951,0
6005,1,59,0,5,13,1,5,1,0,0,3,0,0,0,0,0,0,0,0,2,0,1,20.150000,1046.199951,0.00,0,164.029999,1210.229980,0
